## Parallel fourth-order Runge-Kutta

Given the simplest form of the initial-value problem for ODEs:
$$
y' = f(t, y), \quad y(a) = \alpha,
$$
the classical Runge-Kutta solver is defined as
$$
\begin{align*}
    y^{(n+1)} &= y^{(n)} + \frac{h}{6}(k_1 + 2k_2 + 2k_3 + k_4),\\
    k_1 &= hf\left(t^{(n)}, y^{(n)}\right),\\
    k_2 &= hf\left(t^{(n)} + h/2, y^{(n)} + hk_1/2\right),\\
    k_3 &= hf\left(t^{(n)} + h/2, y^{(n)} + hk_2/2\right),\\
    k_4 &= hf\left(t^{(n)} + h/2, y^{(n)} + hk_3\right),\\
\end{align*}
$$

In accordance with [C. Liu et al. (2011)](https://dl.acm.org/doi/abs/10.1007/978-3-642-25255-6_25), the Runge-Kutta solver can be parallelized as follows:
$$
\begin{align*}
    y_{(n+1)} =& y_n + \frac{h}{6}(k_1^{(n)} + 2k_2^{(n)} + 2k_3^{(n)} + k_4^{(n)}),\\
    k_1^{(n)} =& hf\left(t^{(n)}, y^{(n)}\right),\\
    k_2^{(n)} =& hf\left(t^{(n)} + h/2, y^{(n)} - 3k_1^{(n-1)}/4 + k_2^{(n-1)}/2 + 3k_4^{(n-1)}/4\right),\\
    k_3^{(n)} =& hf\left(t^{(n)} + h/2, y^{(n)} - k_1^{(n-1)} + 2k_2^{(n-1)} - k_3^{(n-1)}/2 \right),\\
    k_4^{(n)} =& hf\left(t^{(n)} + h/2, y^{(n)} + k_1^{(n-1)}/2 + k_4^{(n-1)}/2\right),\\
\end{align*}
$$